In [2]:
from flask import Flask, render_template, request,jsonify
import os
from flask_cors import CORS
import pandas as pd 
import numpy as np 
from sklearn.metrics.pairwise import cosine_similarity

data = pd.read_csv("ml-latest-small/movies.csv")
ratings = pd.read_csv("ml-latest-small/ratings.csv")
data['title'] = data['title'].str.lower()
final_data = ratings.merge(data , on = 'movieId')
average_rating = pd.DataFrame(final_data.groupby('title')['rating'].mean())
average_rating['Total Ratings'] = pd.DataFrame(final_data.groupby('title')['rating'].count()) 
full_data = data.merge(average_rating , on ="title")
full_data = full_data.join(full_data.genres.str.get_dummies("|"))#onehotcoding
cos_sim = cosine_similarity(full_data.iloc[:,5:])
show_data = full_data.iloc[:,:4]

,movieId,title,genres,rating
0,1,<pandas.core.strings.StringMethods object at 0...,Adventure|Animation|Children|Comedy|Fantasy,3.501557
1,2,<pandas.core.strings.StringMethods object at 0...,Adventure|Children|Fantasy,3.501557
2,3,<pandas.core.strings.StringMethods object at 0...,Comedy|Romance,3.501557
3,4,<pandas.core.strings.StringMethods object at 0...,Comedy|Drama|Romance,3.501557
4,5,<pandas.core.strings.StringMethods object at 0...,Comedy,3.501557
...,...,...,...,...
9737,193581,<pandas.core.strings.StringMethods object at 0...,Action|Animation|Comedy|Fantasy,3.501557
9738,193583,<pandas.core.strings.StringMethods object at 0...,Animation|Comedy|Fantasy,3.501557
9739,193585,<pandas.core.strings.StringMethods object at 0...,Drama,3.501557
9740,193587,<pandas.core.strings.StringMethods object at 0...,Action|Animation,3.501557


In [ ]:
app = Flask(__name__)
CORS(app)
@app.route('/movie/', methods=['GET', 'POST'])
def index():
    return render_template('film_get.html')
@app.route('/movie/recommend', methods=['GET', 'POST'])
def data():
    if request.method == 'POST':
        text = request.form['text']
        text = text.lower()
        idx = full_data[full_data['title']==text].index.values
        top5 = np.argsort(cos_sim[idx[0]])[-5:][::-1]
        lst=[]
        for i in top5:
            lst.append(show_data.iloc[i].to_string())
        print(type(lst))    
        return jsonify({"Recommendations":lst})
    
if __name__=="__main__":
    app.run(host='127.0.0.9',port=4455,debug=True,use_reloader=False)       

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.9:4455/ (Press CTRL+C to quit)
127.0.0.1 - - [22/Oct/2020 00:04:33] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [22/Oct/2020 00:04:37] "GET /movie/ HTTP/1.1" 200 -
127.0.0.1 - - [22/Oct/2020 00:04:42] "POST /movie/recommend HTTP/1.1" 200 -


<class 'list'>
